# Gale-Shapley Algorithm

In [1]:
import numpy as np

In [2]:
def gen_pref(n,m):
    guys = np.array([np.random.permutation(m) for x in range(n)])
    girls = np.array([np.random.permutation(n) for x in range(m)])
    return guys, girls

In [3]:
def find_best_guy(girl_p, guys):
    # given a preference of guys `girl_p` and the list of guys proposing to her `guys`, return the best one
    # could probably get a better vectorized implementation, but this suffices
    index = np.argmin([np.where(girl_p == x)[0][0] for x in guys])
    return index, guys[index]

In [4]:
def flip_arr(arr):
    # return a dict whose indices are the elements in `arr` and whose elements are the indices in `arr`
    new_arr = {}
    for i, x in enumerate(arr):
        if not(x in new_arr.keys()):
            new_arr[x] = [i]
        else:
            new_arr[x].append(i)
    return new_arr

In [5]:
def reject(guy_prefs):
    # given a 2D array of multiple guys' preferences of girls, reject their top choices
    n = guy_prefs.shape[0]
    output = np.unique(np.where(guy_prefs == -1)[0], return_counts=True)
    indices = np.zeros(n, dtype=int)
    indices[output[0]] = output[1]
    guy_prefs[np.arange(n), indices] = -1
    return guy_prefs

In [6]:
def gale_shapley(guy_pref, girl_pref):
    n = guy_pref.shape[0]
    m = guy_pref.shape[1]
    best_couples = -1*np.ones(m) # flag values for all
    while -1 in best_couples:
        # have all guys propose to their #1 girl that is left
        # then, girls who were proposed to accept their top offer and reject the rest
        
        # get all the top girls
        output = np.unique(np.where(guy_pref == -1)[0], return_counts=True) # get how many -1s there are by row
        indices = np.zeros(n, dtype=int)
        indices[output[0]] = output[1]
        girls = guy_pref[np.arange(n),indices] # this means that girls[0] is the girl guy #0 proposed to, etc.
        
        proposals = flip_arr(girls)
        for girl, guys in proposals.items():
            index, best_guy = find_best_guy(girl_pref[girl], guys)
            best_couples[girl] = best_guy
            # now reject the others
            guys = np.delete(guys, index)
            guy_pref[guys] = reject(guy_pref[guys])
            
    return best_couples

In [7]:
def is_stable(guy_pref, girl_pref, couple_arrangement):
    for girl, guy in enumerate(couple_arrangement):
        preferred_guys = girl_pref[girl]
        preferred_guys = preferred_guys[:np.where(preferred_guys == guy)[0][0]]
        for guy in preferred_guys:
            # if the guy prefers her too, then it's unstable
            other_girl = np.where(couple_arrangement == guy)[0][0]
            preferred_girls = guy_pref[guy]
            preferred_girls = preferred_girls[:np.where(preferred_girls == other_girl)[0][0]]
            if girl in preferred_girls:
                return False
    return True

In [8]:
guy_pref, girl_pref = gen_pref(10,10)

In [9]:
best_couples = gale_shapley(guy_pref, girl_pref)

In [10]:
is_stable(guy_pref, girl_pref, best_couples)

True

In [11]:
for i, x in enumerate(best_couples):
    x = int(x)
    print('girl {} should be with guy {}'.format(i,x))
    girl_choice = np.where(girl_pref[i] == x)[0][0] + 1
    guy_choice = np.where(guy_pref[x] == i)[0][0] + 1
    print('\tThis was girl {}\'s #{} choice, and it was guy {}\'s #{} choice.'.format(i,girl_choice, x, guy_choice))

girl 0 should be with guy 1
	This was girl 0's #6 choice, and it was guy 1's #6 choice.
girl 1 should be with guy 0
	This was girl 1's #5 choice, and it was guy 0's #1 choice.
girl 2 should be with guy 2
	This was girl 2's #4 choice, and it was guy 2's #3 choice.
girl 3 should be with guy 8
	This was girl 3's #3 choice, and it was guy 8's #4 choice.
girl 4 should be with guy 3
	This was girl 4's #1 choice, and it was guy 3's #1 choice.
girl 5 should be with guy 7
	This was girl 5's #1 choice, and it was guy 7's #2 choice.
girl 6 should be with guy 6
	This was girl 6's #4 choice, and it was guy 6's #2 choice.
girl 7 should be with guy 5
	This was girl 7's #1 choice, and it was guy 5's #1 choice.
girl 8 should be with guy 4
	This was girl 8's #4 choice, and it was guy 4's #2 choice.
girl 9 should be with guy 9
	This was girl 9's #6 choice, and it was guy 9's #2 choice.
